In [47]:
import pandas as pd
import matplotlib.pyplot as plt

import pyspark

import sklearn as skl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords, wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [49]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read CSV into DataFrame") \
    .getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [53]:
file_path = 'Course Relevance Dataset.csv'

# Read CSV into DataFrame
course_data_df = spark.read.csv(file_path, header=True, inferSchema=True)

AttributeError: 'property' object has no attribute 'csv'

In [45]:
pd.set_option('max_colwidth', None)

In [4]:
# nltk.download('udhr')

In [5]:
# nltk.stem.snowball.demo()

In [6]:
course_data_df = pd.read_csv('Course Relevance Dataset.csv', encoding='latin-1')
course_data_df.head()

,SrNo,Name Of the Program,Type of Course,Code,Need,Description of the need
0,1,BA (Geography),Environmental studies,UGEVS101,"Local, Regional, National and Global","To make student understand the distribution, u..."
1,2,BA (Geography),Environmental studies,UGEVS201,"Local, Regional, National and Global",Examining historical and current environmental...
2,3,BA (Geography),Environmental Science-I,UGESlOl,"Local, Regional, National and Global","To make student understand the distribution, u..."
3,4,BA (Geography),Environmental Science-II,UGES201,"Local, Regional, National and Global",Managing waste is critical for preventing envi...
4,5,BA (Geography),Fundamentals of Geomorphology,UGGEO101,"Local, Regional, National and Global",These geological processes shape the Earth's s...


In [7]:
row_nan_count = course_data_df.isnull().sum()
print(row_nan_count)

SrNo                       0
Name Of the Program        0
Type of Course             0
Code                       0
Need                       1
Description of the need    0
dtype: int64


In [26]:
#  NLTK data - Only need to run once (I think).
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [41]:
# sample = course_data_df['Description of the need'][1]

In [42]:
## Source: https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

# from collections import defaultdict

# lemmatizer = WordNetLemmatizer()

# tokens = word_tokenize(sample.lower())

# tag_map = defaultdict(lambda : wn.NOUN)
# tag_map['J'] = wn.ADJ
# tag_map['V'] = wn.VERB
# tag_map['R'] = wn.ADV

# l = []
# for token, tag in pos_tag(tokens):
#     lemma = lemmatizer.lemmatize(token, tag_map[tag[0]])
#     l.append(lemma)
# print(l)

['examine', 'historical', 'and', 'current', 'environmental', 'movement', 'provide', 'insight', 'into', 'societal', 'response', 'to', 'environmental', 'issue', '.', 'environmental', 'management', 'strategy', 'help', 'student', 'understand', 'how', 'to', 'balance', 'development', 'with', 'conservation', '.', 'student', 'will', 'gain', 'insight', 'into', 'waste', 'disposal', 'technique', ',', 'recycle', 'process', ',', 'and', 'the', 'impact', 'of', 'improper', 'waste', 'management', 'on', 'the', 'environment']


In [43]:
# https://www.nltk.org/_modules/nltk/stem/wordnet.html
lemmatizer = WordNetLemmatizer()

# Process text
def preprocess_text(text):
    #Tokenize text
    tokens = word_tokenize(text.lower())
    # Remove stopwords and numbers
    tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # print(tokens)
    return ' '.join(tokens)

# Run function and add to processed column
course_data_df['processed_description'] = course_data_df['Description of the need'].apply(preprocess_text)

course_data_df[['Description of the need', 'processed_description']].head()

,Description of the need,processed_description
0,"To make student understand the distribution, u...",make student understand distribution utilizati...
1,Examining historical and current environmental...,examining historical current environmental mov...
2,"To make student understand the distribution, u...",make student understand distribution utilizati...
3,Managing waste is critical for preventing envi...,managing waste critical preventing environment...
4,These geological processes shape the Earth's s...,geological process shape earth surface underst...


In [22]:
print(lemmatizer.lemmatize('universal'))

universal


In [47]:
course_data_df.head()

,SrNo,Name Of the Program,Type of Course,Code,Need,Description of the need,processed_description
0,1,BA (Geography),Environmental studies,UGEVS101,"Local, Regional, National and Global","To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment",make student understand distribution utilization conservation natural resource crucial sustainable development involves studying earth renewable resource impact environment
1,2,BA (Geography),Environmental studies,UGEVS201,"Local, Regional, National and Global","Examining historical and current environmental movements provides insight into societal responses to environmental issues. Environmental management strategies help students understand how to balance development with conservation. Students will gain insights into waste disposal techniques, recycling processes, and the impact of improper waste management on the environment",examining historical current environmental movement provides insight societal response environmental issue environmental management strategy help student understand balance development conservation student gain insight waste disposal technique recycling process impact improper waste management environment
2,3,BA (Geography),Environmental Science-I,UGESlOl,"Local, Regional, National and Global","To make student understand the distribution, utilization, and conservation of natural resources is crucial for sustainable development It involves studying the Earth's renewable and non-renewable resources and their impact on the\nenvironment",make student understand distribution utilization conservation natural resource crucial sustainable development involves studying earth renewable resource impact environment
3,4,BA (Geography),Environmental Science-II,UGES201,"Local, Regional, National and Global","Managing waste is critical for preventing environmental degradation. Students need to understand waste generation, treatment, and disposal methods to contribute to sustainable waste management practices.Students will comprehend the causes and effects of pollution, as well as the regulatory frameworks and policies aimed at environmental conservation.",managing waste critical preventing environmental degradation student need understand waste generation treatment disposal method contribute sustainable waste management comprehend cause effect pollution well regulatory framework policy aimed environmental conservation
4,5,BA (Geography),Fundamentals of Geomorphology,UGGEO101,"Local, Regional, National and Global","These geological processes shape the Earth's surface. Understanding endogenic (internal) and exogenic (external) processes helps in comprehending landforms, earthquakes,\nand erosion.",geological process shape earth surface understanding endogenic internal exogenic external process help comprehending landforms earthquake erosion


In [63]:
corpus = course_data_df['processed_description']

tfidf_vectorizer = TfidfVectorizer()
count_vectorizer = CountVectorizer()

x = tfidf_vectorizer.fit_transform(corpus)
tfidf_vectorizer.get_feature_names_out()

array(['abide', 'ability', 'able', ..., 'wsns', 'year', 'yield'],
      dtype=object)

In [64]:
x.shape

(879, 1907)

In [67]:
corpus2 = count_vectorizer.fit_transform(corpus)
print(count_vectorizer.get_feature_names_out())

['abide' 'ability' 'able' ... 'wsns' 'year' 'yield']


In [69]:
cv_df = pd.DataFrame(corpus2.toarray(), columns=count_vectorizer.get_feature_names_out())

In [70]:
cv_df

,abide,ability,able,abnormal,abstract,academia,academic,acceptance,access,accessible,...,worker,workforce,working,world,worldwide,write,writing,wsns,year,yield
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
875,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
##